### NLP team Data Challenge!!!
Hello interns! This is your first day working at BigBlueSteam Co!

Until 17.00 o’clock you have to use your Steam Game Review Dataset to create a model that predicts from text whether the user recommends a game title or not.

Until now our platform hosts reviews and game publishers accuse us of hosting comments that are often  unjustifiable and not based on evidence but on rage. Our company will use your model real-time so positive comments can be publicly available instantly. On the other hand, negative ones have to be detected in order to be reviewed before becoming public so we can avoid excessive language and unjustifiable accusations.

# Data Understanding

In [ ]:
#!pip install better-profanity

In [ ]:
#Imports
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
from ipywidgets import interact, fixed
import ipywidgets as widgets
from io import BytesIO
import re
from re import sub
from PIL import Image
import textwrap
import urllib.request
from wordcloud import WordCloud, ImageColorGenerator
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
train = pd.read_csv('.\\data\\train_gr\\train.csv')
train.head(5)

In [ ]:
games = pd.read_csv('.\\data\\train_gr\\game_overview.csv')
games.head(5)

In [ ]:
test = pd.read_csv('.\\data\\test_gr\\test.csv')
test.head(5)

# Data Preparation

In [ ]:
# function for performing a summarising, preliminary EDA
def initial_eda(df):
    """ Prints the dataFrames's column names, data types, 
    number of distinct values, number of missing values
    @param df: pandas DataFrame
    """
    if isinstance(df, pd.DataFrame):
        total_na = df.isna().sum().sum()
        print("Dimensions : %d rows, %d columns" % (df.shape[0], df.shape[1]))
        print("Total NA Values : %d " % (total_na))
        print("%35s %20s   %10s %10s" % ("Column Name", "Data Type", "#Distinct", "NA Values"))
        col_name = df.columns
        dtyp = df.dtypes
        uniq = df.nunique()
        na_val = df.isna().sum()
        for i in range(len(df.columns)):
            print("%35s %20s   %10s %10s" % (col_name[i], dtyp[i], uniq[i], na_val[i]))
    else:
        print("Expect a DataFrame but got a %15s" % (type(df)))

In [ ]:
initial_eda(games)

In [ ]:
initial_eda(train)

In [ ]:
initial_eda(test)

In [ ]:
# cleaning
def clean_text(comment):
    return ''.join(re.sub(r"(@[A-Za-z0-9]+)|(http\S+)|(\$[A-Za-z0-9]+)|([0-9]+)","",comment))

In [ ]:
def remove_special_chars(comment):# unrolls hastags and also removes symbols
    for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
        comment.replace(remove, "", inplace=True)
    return comment

In [ ]:
badwords1 = pd.read_csv('badwords.txt', header=None)
badwords2 = pd.read_csv('bad-words.csv', header=None)

In [ ]:
bad = badwords1.append(badwords2)
bad.drop_duplicates(inplace = True)

In [ ]:
def remove_stopwords(input_text):
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list 
                                              or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 

## 1. Before anything else we need at some point all bad-language to be removed.

In [ ]:
def preprocessing(df):
    df["clean_text"]=df['user_review'].apply(lambda row: remove_stopwords(row))
    df["clean_text"]=df['clean_text'].apply(lambda row: clean_text(row))
    remove_special_chars(df.clean_text)
    df["badwordcount"] = df['user_review'].apply(
        lambda comment: sum(comment.count(str(w)) for w in bad))
    df['num_words'] = df['user_review'].apply(
            lambda comment: len(comment.split()))
    df['num_chars'] = df['user_review'].apply(len)
    df["normword_badwords"] = df["badwordcount"]/df['num_words']
    
    return df

In [ ]:
train = preprocessing(train)
train

In [ ]:
train['profanity_flag'] = train['clean_text'].apply(lambda x: profanity.contains_profanity(x))

In [ ]:
train['profanity_flag']

## 2.    It would be a great idea to accompany your script with a spell checking mechanism. (Demonstrate  it on single reviews not all dataset it may too take looong!!!)

## 3. Try to capture the comments with the most intense feelings of joy or anger.    Provide us with a model and script to run you classification from command line (you could keep a portion of data as a new csv to try it as external data)

## 4. Last but not least, any analysis on the reviews would be valuable and highly appreciated!


In [ ]:
# angel ................